In [49]:
import json
import datetime
import requests
import pandas as pd
from openai import OpenAI
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import os
import re

## Преобразование в различные форматы

In [3]:
def save_to_other_formats(sub_directory, ext, sheet_name):
    df = pd.read_excel(f"./dataset/{sub_directory}/input.{ext}", sheet_name)
    df = df.replace("", pd.NA)
    df = df.dropna(how="all")
    df = df.fillna("")
    df = df.map(
        lambda x: int(x) if isinstance(x, float) and x.is_integer()
        else round(x, 2) if isinstance(x, (int, float))
        else x
    )
    df = df.map(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
    df = df.map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)
    df = df.rename(columns=lambda x: "" if str(x).startswith("Unnamed") else x)
    df = df.map(lambda x: str(x).strip())
    
    df.to_csv(f"./dataset/{sub_directory}/input.csv", index=False)
    df.to_html(f"./dataset/{sub_directory}/input.html", index=False)
    df.to_markdown(f"./dataset/{sub_directory}/input.md", index=False)

In [11]:
# save_to_other_formats("ВРП", "xlsx", 2)
# save_to_other_formats("Ввод жилых домов", "xls", "сентябрь2025")
# save_to_other_formats("Строительство", "xlsx", "Млн.рублей")
# save_to_other_formats("Индекс производства (оперативные данные)", "xls", "Данные")
# save_to_other_formats("Средний возраст", "xlsx", "1.11.")
# save_to_other_formats("Численность и состав рабочей силы", "xlsx", "Sheet1")
# save_to_other_formats("Число родившихся", "xlsx", "t1_2")

# save_to_other_formats("Cоциально-экономическое положение", "xlsx", "1")
# save_to_other_formats("ВВП", "xlsx", "2")

save_to_other_formats("Инвестиции", "xlsx", "млн. рублей")

save_to_other_formats("Индекс потребительских цен (за рассматриваемый период)", "xls", "Данные")
save_to_other_formats("Индекс потребительских цен (к декабрю предыдущего года)", "xls", "Данные")

save_to_other_formats("Непродовольственные товары (за рассматриваемый период)", "xls", "Данные")
save_to_other_formats("Непродовольственные товары (к декабрю предыдущего года)", "xls", "Данные")

save_to_other_formats("Ожидаемая продолжительность жизни", "xls", "Данные")

save_to_other_formats("ОКВЭД", "xlsx", "1")

save_to_other_formats("Потребность работодателей", "xlsx", "Sheet1")

save_to_other_formats("Продовольственные товары (за рассматриваемый период)", "xls", "Данные")
save_to_other_formats("Продовольственные товары (к декабрю предыдущего года)", "xls", "Данные")

save_to_other_formats("Просроченная задолженность", "xlsx", "общая задолженность_с 2019")

save_to_other_formats("Реальная заработная плата", "xls", "Данные")

save_to_other_formats("Торговля и услуги", "xlsx", "Млн. рублей")

save_to_other_formats("Уровень бедности", "xlsx", "по РФ и субъектам РФ 1995-2024")

save_to_other_formats("Уровень зарегистрированной безработицы", "xlsx", "Sheet1")

save_to_other_formats("Численность не занятых", "xlsx", "Sheet1")

save_to_other_formats("Численность постоянного населения", "xls", "Данные")

## Тестирование LLM

In [12]:
load_dotenv()
token = os.getenv("HF_TOKEN")
key = os.getenv("OR_KEY")

In [48]:
def strip_json_fence(text: str) -> str:
    return re.sub(r"^```(?:json)?\s*|\s*```$", "", text.strip(), flags=re.IGNORECASE)

In [ ]:
def call_llm(base_url, api_key, model, system_message, prompt):
    client = OpenAI(
        base_url=base_url,
        api_key=api_key
    )
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
        # temperature=0,
    )
    
    return completion.choices[0].message.content

In [81]:
def get_differences(model_output, expected_data):
    ref_dict = {(item['subject'], item['year']): item for item in expected_data}
    model_dict = {(item['subject'], item['year']): item for item in model_output}
    
    differences = []
    for key, ref_item in ref_dict.items():
        model_item = model_dict.get(key)
        if model_item is None:
            differences.append(('Ключ отсутствует в результате', key))
        elif model_item != ref_item:
            differences.append(('Есть различия', key, model_item, ref_item))

    return differences

In [82]:
def save_json_data(directory, data):
    os.makedirs(directory, exist_ok=True)

    filename = os.path.join(
        directory,
        f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
    )
    
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)
        print(f"result saved to {filename}")

In [85]:
def call_llm_save_result_and_validate(base_url, api_key, model, system_message, prompt, save_dir, expected_data):
    print(model)
    
    content = call_llm(base_url, api_key, model, system_message, prompt)
    
    print(f"response content len: {len(content)}")

    try:
        result = json.loads(strip_json_fence(content))
    except ZeroDivisionError:
        print("response is not valid json")
        return result
    
    for item in result:
        if 'month' not in item:
            item['month'] = None
        if 'day' not in item:
            item['day'] = None
    
    # Создаем путь ./save_dir/outputs/<model_name>
    model_name_safe = model.split(":")[0].replace("/", "-")
    directory = os.path.join(save_dir, "outputs", model_name_safe)
    os.makedirs(directory, exist_ok=True)
    
    save_json_data(directory, result)

    differences = get_differences(result, expected_data)
    print(f'Найдено {len(differences)} отличий')

    return result

### ВРП | HF | gpt-oss-120b

In [73]:
system = f"""
Твоя задача - преобразовать входные данные в формате CSV в формат JSON. В результате должен получиться массив объектов с полями:
- subject (субъект Российской федерации)
- year (год)
- month (месяц, если есть)
- day (день, если есть)
- value (число на пересечении региона и периода)

Сохраняй названия субъектов и городов точно как в исходном CSV, без изменений/сокращений, без добавления или удаления пробелов.

Выведи только валидный JSON (полностью), без объяснений.
"""

with open("./dataset/ВРП/input.csv", "r", encoding="utf-8") as file:
    data = file.read()

with open("./dataset/ВРП/outputs/valid.json", "r", encoding="utf-8") as file:
    expected_data = json.loads(file.read())

In [84]:
result = call_llm_save_result_and_validate(
    base_url="https://router.huggingface.co/v1",
    api_key=token,
    model="openai/gpt-oss-120b:groq",
    system_message=system,
    prompt=data,
    save_dir="./dataset/ВРП",
    expected_data=expected_data
)

response content len: 168
result saved to ./dataset/ВРП\outputs\openai-gpt-oss-120b\2026-01-12_18-40-47.json
Найдено 768 отличий


In [63]:
# валидные данные
with open("./dataset/ВРП/outputs/valid.json", "r", encoding="utf-8") as file:
    valid = json.loads(file.read())
    
# Создаем словарь для быстрого поиска
ref_dict = {(item['subject'], item['year']): item for item in valid}
model_dict = {(item['subject'], item['year']): item for item in result}

differences = []
for key, ref_item in ref_dict.items():
    model_item = model_dict.get(key)
    if model_item is None:
        differences.append(('Ключ отсутствует в результате', key))
    elif model_item != ref_item:
        differences.append(('Есть различия', key, model_item, ref_item))

print(f'Найдено {len(differences)} отличий')

Найдено 32 отличий


In [64]:
differences

[('Ключ отсутствует в результате', ('Ненецкий автономный округ', 2016)),
 ('Ключ отсутствует в результате', ('Ненецкий автономный округ', 2017)),
 ('Ключ отсутствует в результате', ('Ненецкий автономный округ', 2018)),
 ('Ключ отсутствует в результате', ('Ненецкий автономный округ', 2019)),
 ('Ключ отсутствует в результате', ('Ненецкий автономный округ', 2020)),
 ('Ключ отсутствует в результате', ('Ненецкий автономный округ', 2021)),
 ('Ключ отсутствует в результате', ('Ненецкий автономный округ', 2022)),
 ('Ключ отсутствует в результате', ('Ненецкий автономный округ', 2023)),
 ('Ключ отсутствует в результате',
  ('Архангельская область без Ненецкого автономного округа', 2016)),
 ('Ключ отсутствует в результате',
  ('Архангельская область без Ненецкого автономного округа', 2017)),
 ('Ключ отсутствует в результате',
  ('Архангельская область без Ненецкого автономного округа', 2018)),
 ('Ключ отсутствует в результате',
  ('Архангельская область без Ненецкого автономного округа', 2019)),


### Потребность работодателей | OR | mistral-small-3.1-24b-instruct

In [47]:
system = f"""
Твоя задача - преобразовать входные данные в формате CSV в формат JSON. В результате должен получиться массив объектов с полями:
- subject (субъект Российской федерации)
- year (год)
- month (месяц - число от 1 до 12, если есть)
- day (день, если есть)
- value (число на пересечении региона и периода)

Сохраняй названия субъектов и городов точно как в исходном CSV, без изменений/сокращений, без добавления или удаления пробелов.

Выведи только валидный JSON (полностью), без объяснений.
"""

with open("./dataset/Потребность работодателей/input.csv", "r", encoding="utf-8") as file:
    data = file.read()

In [56]:
call_llm_and_save_result(
    base_url="https://openrouter.ai/api/v1",
    api_key=key,
    model="mistralai/mistral-small-3.1-24b-instruct:free",
    system_message=system,
    prompt=data,
    save_dir="./dataset/Потребность работодателей"
)

response content len: 29141
result saved to ./dataset/Потребность работодателей\outputs\mistralai-mistral-small-3.1-24b-instruct\2026-01-12_17-59-17.json


In [57]:
API_URL = "https://router.huggingface.co/featherless-ai/v1/completions"
headers = {
    "Authorization": f"Bearer {token}",
}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

output = query({
    "model": "meta-llama/Meta-Llama-3-8B-Instruct",
    "prompt": prompt
})

print(output['choices'][0]['text'])

{'id': '3ae20cde-b883-4271-b44c-6c869176cde7', 'object': 'text_completion', 'created': 1763980958480, 'model': 'meta-llama/Meta-Llama-3-8B-Instruct', 'choices': [{'index': 0, 'text': '```\n[\n  {\n    "subject": "Валовой региональный продукт по субъектам Российской Федерации",\n    "year": 1998,\n    "value": 2251977.5\n  },\n  {\n    "subject": "Центральный федеральный округ",\n    "year": 1998,\n    "value": 634372.0\n  },\n  {\n    "subject": "Белгородская область",\n    "year": 1998,\n    "value": 18245.5\n  },\n  {\n    "subject": "Брянская область",\n    "year": 1998,\n    "value": 11051.3\n  },\n  {\n    "subject": "Владимирская область",\n    "year": 1998,\n    "value": 14936.9\n  },\n  {\n    "subject": "Воронежская область",\n    "year": 1998,\n    "value": 22381.9\n  },\n  {\n    "subject": "Ивановская область",\n    "year": 1998,\n    "value": 8278.4\n  },\n  {\n    "subject": "Калужская область",\n    "year": 1998,\n    "value": 10097.3\n  },\n  {\n    "subject": "Валовой 

In [66]:
API_URL = "https://router.huggingface.co/featherless-ai/v1/completions"
headers = {
    "Authorization": f"Bearer {token}",
}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

output = query({
    "model": "meta-llama/Meta-Llama-3-8B",
    "prompt": prompt
})

print(output['choices'][0]['text'])

In [38]:
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=token,
)

completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1:novita",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)

print(completion.choices[0].message)

ChatCompletionMessage(content='<think>\nМы замечаем, что в CSV представлены данные по годам (1998-2002) и по субъектам РФ.\n Нам нужно преобразовать каждую строку с субъектом в несколько объектов (по одному на каждый год).\n Структура:\n   Первые две строки - это заголовки, которые мы пропустим.\n   Первая строка данных (после пропуска двух строк) - это заголовок колонок: пустая, затем годы.\n   Следующие строки - данные: в первом столбце название субъекта, затем значения по годам.\n\n Нам нужен массив объектов, где каждый объект соответствует одной записи для субъекта в конкретном году.\n\n Поля:\n   subject: название субъекта (берем из первого столбца строки)\n   year: год (берем из заголовка столбца)\n   value: значение в соответствующем году (берем из ячейки, соответствующей году и строке субъекта)\n\n Месяц и день в данных отсутствуют, поэтому в объекте оставляем поля month и day как null или не включаем, если не указано.\n Однако по условию задачи требуется включать поля month и 

In [6]:
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=token,
)

completion = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2:featherless-ai",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)

print(completion.choices[0].message)

ChatCompletionMessage(content=' [\n [{"subject": "Центральный федеральный округ", "year": 1998, "value": 634372.0},\n  {"subject": "Центральный федеральный округ", "year": 1999, "value": 1190894.4},\n  {"subject": "Центральный федеральный округ", "year": 2000, "value": 1841498.9},\n  {"subject": "Центральный федеральный округ", "year": 2001, "value": 2243525.0},\n  {"subject": "Центральный федеральный округ", "year": 2002, "value": 2878664.5},\n  {"subject": "Центральный федеральный округ", "year": 2003, "value": 3577142.5},\n  {"subject": "Центральный федеральный округ", "year": 2004, "value": 4617086.1},\n  {"subject": "Центральный федеральный округ", "year": 2005, "value": 6278359.2},\n  {"subject": "Центральный федеральный округ", "year": 2006, "value": 7965169.5},\n  {"subject": "Белгородская область", "year": 1998, "value": 18245.5},\n  {"subject": "Белгородская область", "year": 1999, "value": 32060.6},\n  <!-- OMITTED: REPEATED ENTRIES FOR OTHER SUBJECTS -->\n  {"subject": "Кур

In [8]:
prompt = """
Есть таблица со следующими заголовками колонок:
| 2024   |         |      |        |     |      |      |        |          |         |        |         | 2025   |         |      |
| ------ | ------- | ---- | ------ | --- | ---- | ---- | ------ | -------- | ------- | ------ | ------- | ------ | ------- | ---- |
| январь | февраль | март | апрель | май | июнь | июль | август | сентябрь | октябрь | ноябрь | декабрь | январь | февраль | март |

Твоя задача смерджить заголовки и вернуть массив значений в виде:
```
["01-2024", "02-2024", ...]
```

Выведи только валидный JSON (полностью) с итоговым массивом, без объяснений.
"""

In [9]:
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=token,
)

completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-Coder-32B-Instruct:nscale",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)

print(completion.choices[0].message)

InternalServerError: <!DOCTYPE html>
<html class="" lang="en">
<head>
    <meta charset="utf-8" />
    <meta
            name="viewport"
            content="width=device-width, initial-scale=1.0, user-scalable=no"
    />
    <meta
            name="description"
            content="We're on a journey to advance and democratize artificial intelligence through open source and open science."
    />
    <meta property="fb:app_id" content="1321688464574422" />
    <meta name="twitter:card" content="summary_large_image" />
    <meta name="twitter:site" content="@huggingface" />
    <meta
            property="og:title"
            content="Hugging Face - The AI community building the future."
    />
    <meta property="og:type" content="website" />

    <title>Hugging Face - The AI community building the future.</title>
    <style>
        body {
            margin: 0;
        }

        main {
            background-color: white;
            min-height: 100vh;
            padding: 7rem 1re